In [17]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import dlib
from keras.models import load_model


In [12]:
file = np.genfromtxt('data/train_data.csv', delimiter=',')
file = np.delete(file, (0), axis = 0)

In [13]:
file[0].shape

(4,)

In [14]:
feature = file[:,:-1].astype(np.float32)
label = file[:, -1].astype(np.float32)

In [15]:
knn = cv2.ml.KNearest_create()
knn.train(feature, cv2.ml.ROW_SAMPLE, label)

True

#### 계산해야 하는 feature
* degree, Diff_X, Diff_Y

In [16]:
def Nose_Point(result, image_width, image_height) :
    Nose_X = result.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * image_width
    Nose_Y = result.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y * image_height

    return Nose_X, Nose_Y

def Compute_Neck(result, image_width, image_height) :
    L_Shoulder_X = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * image_width
    R_Shoulder_X = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * image_width
    Neck_X = (L_Shoulder_X + R_Shoulder_X) / float(2.0)
    
    R_Shoulder_Y = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * image_height
    L_Shoulder_Y = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * image_height
    Neck_Y = (L_Shoulder_Y + R_Shoulder_Y) / float(2.0)

    return Neck_X, Neck_Y

def Compute_Diff(Nose_X, Nose_Y, Neck_X, Neck_Y):
    Diff_X = Neck_X - Nose_X
    Diff_Y = Neck_Y - Nose_Y
    return Diff_X, Diff_Y

def Comput_Degree(Nose_X, Nose_Y, Neck_X, Neck_Y):
    AB = np.sqrt((Nose_X - Neck_X) ** 2 + (Nose_Y - Neck_Y) ** 2)
    AC = np.abs(Nose_X - Neck_X)
    cos = AC / AB
    degree = np.degrees(cos)
    return degree

def Point_Eye(result, image_width, image_height):
    R_Eye_X = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x * image_width
    R_Eye_Y = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y * image_height

    L_Eye_X = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE].x * image_width
    L_Eye_Y = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE].y * image_height
    return int(R_Eye_X), int(R_Eye_Y), int(L_Eye_X), int(L_Eye_Y)

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect


#### eye detect model load

In [19]:
model = load_model('models/eye_detecting.h5')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26, 34, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 34, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 17, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 17, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 8, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 8, 128)         73856 

In [21]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('dlib_eye/shape_predictor_68_face_landmarks.dat')

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture("test.mp4")
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
    
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      break
    

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_height, image_width, _ = image.shape
    result = pose.process(image)
    
    # compute degree and diff
    try:
        Nose_X, Nose_Y = Nose_Point(result, image_width, image_height)
        Neck_X, Neck_Y = Compute_Neck(result, image_width, image_height)
        Diff_X, Diff_Y = Compute_Diff(Nose_X, Nose_Y, Neck_X, Neck_Y)
        degree = Comput_Degree(Nose_X, Nose_Y, Neck_X, Neck_Y)
        data = np.array([degree, Diff_X, Diff_Y], dtype = np.float32)
        data = data.reshape(1,3)
    
        ret, label, neighbours, dist = knn.findNearest(data, 2)    
    except:
        continue
    
    try:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        faces = detector(gray)
        for face in faces:
            shapes = predictor(gray, face) 
            shapes = face_utils.shape_to_np(shapes)

            eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
            eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

            eye_img_l = cv2.resize(eye_img_l, dsize=(34,26))
            eye_img_r = cv2.resize(eye_img_r, dsize=(34,26))
            eye_img_r = cv2.flip(eye_img_r, flipCode=1)
        
            eye_img_l3 = np.reshape(eye_img_l ,(26,34,1))
            eye_img_r3 = np.reshape(eye_img_r ,(26,34,1))
            cv2_imshow(eye_img_l)
            cv2_imshow(eye_img_r)
            r = model.predict(np.expand_dims(eye_img_r3, axis = 0))
            l = model.predict(np.expand_dims(eye_img_l3, axis = 0))
        
            if (int(r[0])==1 or int(l[0]) == 1) : category = 0
            if (int(r[0])==0 and int(l[0]) == 0) : category = 1
    except:
        continue

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        result.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    if label == 1:
        text = "Drowsy"
    else:
        text = "NonDorwsy"
    
    cv2.putText(image, text, (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('MediaPipe Pose', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

Ignoring empty camera frame.


## 전체 코드

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd

file = np.genfromtxt('data/train_data.csv', delimiter=',')
file = np.delete(file, (0), axis = 0)

feature = file[:,:-1].astype(np.float32)
label = file[:, -1].astype(np.float32)

knn = cv2.ml.KNearest_create()
knn.train(feature, cv2.ml.ROW_SAMPLE, label)

def Nose_Point(result, image_width, image_height) :
    Nose_X = result.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * image_width
    Nose_Y = result.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y * image_height

    return Nose_X, Nose_Y

def Compute_Neck(result, image_width, image_height) :
    L_Shoulder_X = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * image_width
    R_Shoulder_X = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * image_width
    Neck_X = (L_Shoulder_X + R_Shoulder_X) / float(2.0)
    
    R_Shoulder_Y = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * image_height
    L_Shoulder_Y = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * image_height
    Neck_Y = (L_Shoulder_Y + R_Shoulder_Y) / float(2.0)

    return Neck_X, Neck_Y

def Compute_Diff(Nose_X, Nose_Y, Neck_X, Neck_Y):
    Diff_X = Neck_X - Nose_X
    Diff_Y = Neck_Y - Nose_Y
    return Diff_X, Diff_Y

def Comput_Degree(Nose_X, Nose_Y, Neck_X, Neck_Y):
    AB = np.sqrt((Nose_X - Neck_X) ** 2 + (Nose_Y - Neck_Y) ** 2)
    AC = np.abs(Nose_X - Neck_X)
    cos = AC / AB
    degree = np.degrees(cos)
    return degree

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_height, image_width, _ = image.shape
    result = pose.process(image)
    
    # compute degree and diff
    Nose_X, Nose_Y = Nose_Point(result, image_width, image_height)
    Neck_X, Neck_Y = Compute_Neck(result, image_width, image_height)
    Diff_X, Diff_Y = Compute_Diff(Nose_X, Nose_Y, Neck_X, Neck_Y)
    degree = Comput_Degree(Nose_X, Nose_Y, Neck_X, Neck_Y)
    data = np.array([degree, Diff_X, Diff_Y], dtype = np.float32)
    data = data.reshape(1,3)
    
    ret, label, neighbours, dist = knn.findNearest(data, 2)    
    

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        result.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    if label == 1:
        text = "Drowsy"
    else:
        text = "NonDorwsy"
    
    cv2.putText(image, text, (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('MediaPipe Pose', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

In [5]:
import cv2
cam = cv2.VideoCapture(0)

while True:
    ret_val, img = cam.read() # 캠 이미지 불러오기
 
    cv2.imshow("Cam Viewer",img) # 불러온 이미지 출력하기
    if cv2.waitKey(1) == 27:
        break  # esc to quit
